In [1]:
# get stuff
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sys, os
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from base.base_func import fill_nans, make_prediction, roc_score, make_cats, base_clf_stack, adv_clf_stack
from base.outliners import plot_outlier, replace_outlier

df_train = pd.read_csv("train.csv", index_col="PassengerId")
df_test = pd.read_csv("test.csv", index_col="PassengerId")

df_train.shape, df_test.shape

((891, 11), (418, 10))

In [2]:
for df in (df_train, df_test):

    # convert text to categories of int
    df = make_cats(df)
    # fill nan values with -999
    fill_nans(df, filler=-999)



df_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,108,1,22.0,1,0,523,7.2500,147,2
2,1,1,190,0,38.0,1,0,596,71.2833,81,0
3,1,3,353,0,26.0,0,0,669,7.9250,147,2
4,1,1,272,0,35.0,1,0,49,53.1000,55,2
5,0,3,15,1,35.0,0,0,472,8.0500,147,2


In [9]:
# split data, get models, produce submission file

X_train = df_train.drop("Survived", axis=1)
Y_train = df_train["Survived"]
X_test  = df_test.copy()
X_train.shape, Y_train.shape, X_test.shape

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

#y_pred = adv_clf_stack(X_train, Y_train, X_test, clf_models=classifiers)
y_pred = base_clf_stack(X_train, Y_train, X_test)
submission = pd.DataFrame({
    "PassengerId": df_test.index,
    "Survived": y_pred,
})

submission.to_csv('stacked_all.csv', index=False)

task:       [regression]
metric:     [r2_score]
mode:       [oof_pred_bag]
n_models:   [3]

model 0:    [ExtraTreesClassifier]
    fold 0: [0.02465753]
    fold 1: [0.06977401]
    fold 2: [0.35154827]
    fold 3: [0.22618591]
    ----
    MEAN:   [0.16804143] + [0.12968324]
    FULL:   [0.17421379]

model 1:    [RandomForestClassifier]
    fold 0: [-0.02178735]
    fold 1: [0.12005650]
    fold 2: [0.32660782]
    fold 3: [0.13514896]
    ----
    MEAN:   [0.14000648] + [0.12391418]
    FULL:   [0.14450925]

model 2:    [GradientBoostingClassifier]
    fold 0: [-0.09145466]
    fold 1: [0.24576271]
    fold 2: [0.35154827]
    fold 3: [0.27170439]
    ----
    MEAN:   [0.19439018] + [0.16957488]
    FULL:   [0.19797742]

S_train shape: (712, 3)
